For each model:
1. Load model
2. Score validation set
3. Compute FLOPS and check file size and nb parameters
4. Get accuracy
5. Write metrics to Dataset
6. Prune last layer using L1 (10-20-30-40-50-60 % of the filters), get metrics and score, write to file - Retrain 1 epoch and score
7. Prune last layer using k-means random (10-20-30-40-50-60 % of the filters), get metrics and score, write to file
8. Prune last layer using k-means centroid (10-20-30-40-50-60 % of the filters)
9. Prune last layer using APoZ (10-20-30-40-50-60 % of the filters), get metrics and score, write to file
10. Prune all layers using L1 (10-20-30-40-50-60 % of the filters), get metrics and score, write to file
10. Prune all layers using APoZ (10-20-30-40-50-60 % of the filters), get metrics and score, write to file


In [6]:
import warnings
warnings.filterwarnings('ignore')

In [7]:
import pruning_utils
import keras
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import numpy as np
import os

In [11]:
class PruningExperiment:
    
    def __init__(self, model_path, methods, percentages, retrain=False, layer=None):
        model_file = model_path.split('/')[-1]
        model_name = '_'.join(model_file.split('_')[:2])
        self.model_path = model_path
        self.methods = methods
        self.percentages = percentages
        self.model_name = model_name
        self.parent_dir = '/'.join(model_path.split('/')[:-1])
        self.retrain=retrain
        self.layer=layer
        
    
    def run(self):
        opt = keras.optimizers.RMSprop(lr=0.0001, decay=1e-6)
        df = pd.read_csv('/home/ec2-user/Telecom/experiments/results.csv')
        for method in self.methods :
            for perc in self.percentages:
                print('pruning %s with method %s and perc %s' %(self.model_name, method,int(perc*100)))
                # prune model
                model = load_model('/home/ec2-user/Telecom/experiments/saved_models/keras_cnn_imagenette_trained_model.h5')
                model_pruned = pruning_utils.prune_model(model, perc, opt, method, layer)
                del model
                if self.layer or self.layer ==0:
                    pruned_model_path = os.path.join(self.parent_dir,
                                                 self.model_name+'_pruned_%s_layer%s_%s.h5'%(method,
                                                                                             self.layer,
                                                                                             int(perc*100)))
                else:
                    pruned_model_path = os.path.join(self.parent_dir,
                                                 self.model_name+'_pruned_%s_%s.h5'%(method,int(perc*100)))
                    
                model_pruned.save(pruned_model_path)
                print('saving pruned model at %s' % pruned_model_path)
                metrics = pruning_utils.get_model_info(pruned_model_path, model_pruned)
                metrics['accuracy'] = self.score(model_pruned)
                
                #save model
                
                metrics['name']= self.model_name
                metrics['method'] = method
                metrics['pruning'] = int(perc*100)
                metrics['path'] = pruned_model_path
                df=df.append(metrics, ignore_index=True)
                
                if self.retrain :
                    model_pruned_retrain = self.train(model_pruned, 1)
                    del model_pruned
                    retrain_model_path = os.path.join(self.parent_dir,
                                                     self.model_name+'_pruned_retrained_%s_%s.h5'%(method,int(perc*100)))
                    model_pruned_retrain.save(retrain_model_path)
                    print('saving pruned & retrained model at %s' % retrain_model_path)
                    metrics = pruning_utils.get_model_info(retrain_model_path, model_pruned_retrain)
                    metrics['accuracy'] = self.score(model_pruned_retrain)
            
        df.to_csv('/home/ec2-user/Telecom/experiments/results.csv')
        return self
    
    def train(self, model, epochs):
        
        train_path = '/home/ec2-user/Telecom/experiments/data/imagenette2-320/train'

        # The data, split between train and test sets:
        train_datagen = ImageDataGenerator(
                rescale=1./255,    
                featurewise_center=False,  
                samplewise_center=False,  
                featurewise_std_normalization=False,  
                samplewise_std_normalization=False,  
                zca_whitening=False,  
                rotation_range=15,  
                width_shift_range=0.1,  
                height_shift_range=0.1,  
                horizontal_flip=True,  
                vertical_flip=False)
        
        train_generator = train_datagen.flow_from_directory(
        train_path,
        target_size=(160, 160),
        batch_size=32,
        class_mode='categorical')
        
        test_datagen = ImageDataGenerator(rescale=1./255)
        test_dir = "/home/ec2-user/Telecom/experiments/data/imagenette2-320/val"
        validation_generator = test_datagen.flow_from_directory(
                test_dir,
                target_size=(160, 160),
                batch_size=1,
                class_mode='categorical',
                shuffle= False)
        
        model.fit_generator(
          train_generator,
          epochs=epochs,
          validation_data=validation_generator,
          steps_per_epoch = 300,
          validation_steps = 150,
          verbose = 1,
        )
        
        return model
    
    def score(self, model):
        test_datagen = ImageDataGenerator(rescale=1./255)
        test_dir = "/home/ec2-user/Telecom/experiments/data/imagenette2-320/val"
        validation_generator = test_datagen.flow_from_directory(
                test_dir,
                target_size=(160, 160),
                batch_size=1,
                class_mode='categorical',
                shuffle= False)
        
        accuracy = pruning_utils.evaluate(model, validation_generator)
        return accuracy

#### Initialization

In [4]:
model_path='/home/ec2-user/Telecom/experiments/saved_models/keras_cnn_imagenette_trained_model.h5'
results_path = '/home/ec2-user/Telecom/experiments/results.csv'

model_file = model_path.split('/')[-1]
model_name = '_'.join(model_file.split('_')[:2])

In [11]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_dir = "/home/ec2-user/Telecom/experiments/data/imagenette2-320/val"
validation_generator = test_datagen.flow_from_directory(
                test_dir,
                target_size=(160, 160),
                batch_size=1,
                class_mode='categorical',
                shuffle= False)
        
model = load_model(model_path)
metrics = pruning_utils.get_model_info(model_path, model)
metrics['accuracy'] = pruning_utils.evaluate(model, validation_generator)
metrics['name']= model_name
metrics['method'] = 'baseline'
metrics['pruning'] = 0
metrics['path'] = model_path

df = pd.DataFrame(columns=metrics.keys())
df = df.append(metrics, ignore_index=True)
df.to_csv(results_path, index=False)

Found 3925 images belonging to 10 classes.
3925/3925 [==============================] - 87s 22ms/step


#### Pipeline

In [25]:
# L1 all layers
exp = PruningExperiment(model_path, ['l1'], [0.1,0.2,0.3,0.4,0.5])

In [26]:
exp.run()

pruning keras_cnn with method l1 and perc 10
Deleting 96/256 channels from layer: conv2d_68
saving pruned model at /home/ec2-user/Telecom/experiments/saved_models/keras_cnn_pruned_l1_10.h5
Found 3925 images belonging to 10 classes.
3925/3925 [==============================] - 76s 19ms/step
pruning keras_cnn with method l1 and perc 20
Deleting 26/256 channels from layer: conv2d_67
Deleting 166/256 channels from layer: conv2d_68
saving pruned model at /home/ec2-user/Telecom/experiments/saved_models/keras_cnn_pruned_l1_20.h5
Found 3925 images belonging to 10 classes.
3925/3925 [==============================] - 68s 17ms/step
pruning keras_cnn with method l1 and perc 30
Deleting 84/256 channels from layer: conv2d_67
Deleting 204/256 channels from layer: conv2d_68
saving pruned model at /home/ec2-user/Telecom/experiments/saved_models/keras_cnn_pruned_l1_30.h5
Found 3925 images belonging to 10 classes.
3925/3925 [==============================] - 63s 16ms/step
pruning keras_cnn with method l

In [29]:
# Apoz all layers
exp = PruningExperiment(model_path, ['apoz'], [0.1,0.2,0.3,0.4,0.5])
exp.run()

pruning keras_cnn with method apoz and perc 10
Found 944 images belonging to 10 classes.
944/944 [==============================] - 16s 17ms/step
Deleting 1/256 channels from layer: conv2d_67
Deleting 95/256 channels from layer: conv2d_68
saving pruned model at /home/ec2-user/Telecom/experiments/saved_models/keras_cnn_pruned_apoz_10.h5
Found 3925 images belonging to 10 classes.
3925/3925 [==============================] - 78s 20ms/step
pruning keras_cnn with method apoz and perc 20
Found 944 images belonging to 10 classes.
944/944 [==============================] - 17s 18ms/step
Deleting 1/32 channels from layer: conv2d_61
Deleting 1/256 channels from layer: conv2d_67
Deleting 190/256 channels from layer: conv2d_68
saving pruned model at /home/ec2-user/Telecom/experiments/saved_models/keras_cnn_pruned_apoz_20.h5
Found 3925 images belonging to 10 classes.
3925/3925 [==============================] - 71s 18ms/step
pruning keras_cnn with method apoz and perc 30
Found 944 images belonging 

In [8]:
# Random all layers
exp = PruningExperiment(model_path, ['random'], [0.1,0.2,0.3,0.4,0.5])
exp.run()

pruning keras_cnn with method random and perc 10
Deleting 15/32 channels from layer: conv2d_61
Deleting 13/32 channels from layer: conv2d_62
Deleting 13/64 channels from layer: conv2d_63
Deleting 11/64 channels from layer: conv2d_64
Deleting 7/128 channels from layer: conv2d_65
Deleting 12/128 channels from layer: conv2d_66
Deleting 15/256 channels from layer: conv2d_67
Deleting 10/256 channels from layer: conv2d_68
saving pruned model at /home/ec2-user/Telecom/experiments/saved_models/keras_cnn_pruned_random_10.h5



Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`


Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`


Found 3925 images belonging to 10 classes.
3925/3925 [==============================] - 77s 20ms/step
pruning keras_cnn with method random and perc 20
Deleting 22/32 channels from layer: conv2d_61
Deleting 19/32 channels from layer: conv2d_62
Deleting 21/64 channels from layer: conv2d_63
Deleting 23/64 channels from layer: conv2d_64
Deleting 28/128 channels from layer: conv2d_65
Deleting 27/128 channels from layer: conv2d_66
Deleting 26/256 channels from layer: conv2d_67
Deleting 26/256 channels from layer: conv2d_68
saving pruned model at /home/ec2-user/Telecom/experiments/saved_models/keras_cnn_pruned_random_20.h5
Found 3925 images belonging to 10 classes.
3925/3925 [==============================] - 68s 17ms/step
pruning keras_cnn with method random and perc 30
Deleting 26/32 channels from layer: conv2d_61
Deleting 26/32 channels from layer: conv2d_62
Deleting 32/64 channels from layer: conv2d_63
Deleting 28/64 channels from layer: conv2d_64
Deleting 36/128 channels from layer: conv

In [12]:
# layer 0
exp = PruningExperiment(model_path, ['l1','apoz','random'], [0.1,0.2,0.3,0.4,0.5], layer=0)
exp.run()

pruning keras_cnn with method l1 and perc 10
Deleting 3/32 channels from layer: conv2d_61
saving pruned model at /home/ec2-user/Telecom/experiments/saved_models/keras_cnn_pruned_l1_layer0_10.h5
Found 3925 images belonging to 10 classes.
3925/3925 [==============================] - 105s 27ms/step
pruning keras_cnn with method l1 and perc 20
Deleting 6/32 channels from layer: conv2d_61
saving pruned model at /home/ec2-user/Telecom/experiments/saved_models/keras_cnn_pruned_l1_layer0_20.h5
Found 3925 images belonging to 10 classes.
3925/3925 [==============================] - 104s 26ms/step
pruning keras_cnn with method l1 and perc 30
Deleting 9/32 channels from layer: conv2d_61
saving pruned model at /home/ec2-user/Telecom/experiments/saved_models/keras_cnn_pruned_l1_layer0_30.h5
Found 3925 images belonging to 10 classes.
3925/3925 [==============================] - 104s 26ms/step
pruning keras_cnn with method l1 and perc 40
Deleting 12/32 channels from layer: conv2d_61
saving pruned mode

In [13]:
# last convolution layer
exp = PruningExperiment(model_path, ['l1','apoz','random'], [0.1,0.2,0.3,0.4,0.5], layer=20)
exp.run()

pruning keras_cnn with method l1 and perc 10
Deleting 25/256 channels from layer: conv2d_68
saving pruned model at /home/ec2-user/Telecom/experiments/saved_models/keras_cnn_pruned_l1_layer20_10.h5
Found 3925 images belonging to 10 classes.
3925/3925 [==============================] - 85s 22ms/step
pruning keras_cnn with method l1 and perc 20
Deleting 51/256 channels from layer: conv2d_68
saving pruned model at /home/ec2-user/Telecom/experiments/saved_models/keras_cnn_pruned_l1_layer20_20.h5
Found 3925 images belonging to 10 classes.
3925/3925 [==============================] - 82s 21ms/step
pruning keras_cnn with method l1 and perc 30
Deleting 76/256 channels from layer: conv2d_68
saving pruned model at /home/ec2-user/Telecom/experiments/saved_models/keras_cnn_pruned_l1_layer20_30.h5
Found 3925 images belonging to 10 classes.
3925/3925 [==============================] - 79s 20ms/step
pruning keras_cnn with method l1 and perc 40
Deleting 102/256 channels from layer: conv2d_68
saving pru

In [14]:
# first convolution layer kmeans
exp = PruningExperiment(model_path, ['kmeans_random','kmeans_centroid'], [0.1,0.2,0.3,0.4,0.5], layer=0)
exp.run()

pruning keras_cnn with method kmeans_random and perc 10
Deleting 3/32 channels from layer: conv2d_61
saving pruned model at /home/ec2-user/Telecom/experiments/saved_models/keras_cnn_pruned_kmeans_random_layer0_10.h5
Found 3925 images belonging to 10 classes.
3925/3925 [==============================] - 102s 26ms/step
pruning keras_cnn with method kmeans_random and perc 20
Deleting 6/32 channels from layer: conv2d_61
saving pruned model at /home/ec2-user/Telecom/experiments/saved_models/keras_cnn_pruned_kmeans_random_layer0_20.h5
Found 3925 images belonging to 10 classes.
3925/3925 [==============================] - 100s 26ms/step
pruning keras_cnn with method kmeans_random and perc 30
Deleting 9/32 channels from layer: conv2d_61
saving pruned model at /home/ec2-user/Telecom/experiments/saved_models/keras_cnn_pruned_kmeans_random_layer0_30.h5
Found 3925 images belonging to 10 classes.
3925/3925 [==============================] - 102s 26ms/step
pruning keras_cnn with method kmeans_random

In [15]:
# last convolution layer kmeans
exp = PruningExperiment(model_path, ['kmeans_random','kmeans_centroid'], [0.1,0.2,0.3,0.4,0.5], layer=20)
exp.run()

pruning keras_cnn with method kmeans_random and perc 10
Deleting 25/256 channels from layer: conv2d_68
saving pruned model at /home/ec2-user/Telecom/experiments/saved_models/keras_cnn_pruned_kmeans_random_layer20_10.h5
Found 3925 images belonging to 10 classes.
3925/3925 [==============================] - 91s 23ms/step
pruning keras_cnn with method kmeans_random and perc 20
Deleting 51/256 channels from layer: conv2d_68
saving pruned model at /home/ec2-user/Telecom/experiments/saved_models/keras_cnn_pruned_kmeans_random_layer20_20.h5
Found 3925 images belonging to 10 classes.
3925/3925 [==============================] - 87s 22ms/step
pruning keras_cnn with method kmeans_random and perc 30
Deleting 76/256 channels from layer: conv2d_68
saving pruned model at /home/ec2-user/Telecom/experiments/saved_models/keras_cnn_pruned_kmeans_random_layer20_30.h5
Found 3925 images belonging to 10 classes.
3925/3925 [==============================] - 85s 22ms/step
pruning keras_cnn with method kmeans_